# 🎙️ Project Resonance — RTX 4090 JupyterLab 一键部署

> **适用环境**：JupyterLab | NVIDIA RTX 4090 (24GB VRAM) | CUDA 12.x/13.x  
> **使用方式**：从上到下逐个 Cell 运行（`Shift + Enter`）

---

| 步骤 | 内容 | 预计时间 |
|------|------|----------|
| 1 | 检查 GPU 环境 | < 1 分钟 |
| 2 | 安装依赖 | 3-5 分钟 |
| 3 | 配置训练参数 | < 1 分钟 |
| 4 | 训练模型 | 15-20 分钟 |
| 5 | 启动推理服务 | < 1 分钟 |

## 步骤 1 / 5 — 检查 GPU 环境

In [ ]:
import subprocess, sys

def check_gpu():
    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=name,memory.total,driver_version',
             '--format=csv,noheader'],
            capture_output=True, text=True, check=True
        )
        lines = result.stdout.strip().split('\n')
        for i, line in enumerate(lines):
            parts = [p.strip() for p in line.split(',')]
            gpu_name, mem, driver = parts[0], parts[1], parts[2]
            mem_mb = int(mem.replace(' MiB', ''))
            print(f'  GPU #{i}:  {gpu_name}')
            print(f'  显存:    {mem}')
            print(f'  驱动:    {driver}')
            if mem_mb >= 20000:
                print(f'  ✅ 显存充足，可使用 batch_size=32')
            else:
                print(f'  ⚠️  显存不足 20GB，建议将 BATCH_SIZE 改为 8')
    except FileNotFoundError:
        print('❌ 未找到 nvidia-smi，请确认 GPU 驱动已安装')
        sys.exit(1)

    try:
        import torch
        print(f'\n  PyTorch:  {torch.__version__}')
        print(f'  CUDA:     {torch.version.cuda}')
        print(f'  BF16:     {"✅ 支持" if torch.cuda.is_bf16_supported() else "❌ 不支持（将回退 FP16）"}')
    except ImportError:
        print('  ⚠️  PyTorch 尚未安装，请先运行步骤 2')

print('=' * 50)
print('  GPU 环境检测')
print('=' * 50)
check_gpu()

## 步骤 2 / 5 — 安装 Python 依赖

> ⏳ 首次运行约 3-5 分钟，已安装则几秒完成

In [ ]:
import subprocess, sys

packages = [
    'transformers',
    'peft',
    'datasets',
    'soundfile',
    'librosa',
    'evaluate',
    'jiwer',
    'fastapi',
    'uvicorn[standard]',
    'python-multipart',
    'accelerate',
    'nest-asyncio',   # JupyterLab 异步兼容必须
]

print('📦 正在安装依赖...')
result = subprocess.run(
    [sys.executable, '-m', 'pip', 'install', '-q'] + packages,
    capture_output=True, text=True
)
if result.returncode == 0:
    print('✅ 依赖安装完成')
    print('\n已安装包：')
    for p in packages:
        print(f'  • {p}')
else:
    print('❌ 安装失败：')
    print(result.stderr)

## 步骤 3 / 5 — 配置训练参数

> ✏️ **在此修改路径和参数**，然后运行此 Cell

In [ ]:
import os

# ── ✏️ 按需修改以下配置 ──────────────────────────────────────
DATA_DIR    = '/root/data'            # 训练数据目录（含 labels.json + WAV 文件）
MODEL_DIR   = '/root/lora_output'     # LoRA 权重保存目录
BASE_MODEL  = 'openai/whisper-small'  # 基础模型（small/medium/large-v3）
LANGUAGE    = 'zh'                    # 目标语言
TRAIN_STEPS = 500                     # 训练步数（样本少可降至 200）
BATCH_SIZE  = 32                      # RTX 4090 推荐 32；显存不足改为 8
SERVER_PORT = 8000                    # 推理服务端口
# ────────────────────────────────────────────────────────────

# 自动检测数据并决定是否跳过训练
labels_path = os.path.join(DATA_DIR, 'labels.json')
SKIP_TRAINING = False
RECORD_COUNT  = 0

print('📋 当前配置：')
print(f'  数据目录:  {DATA_DIR}')
print(f'  模型目录:  {MODEL_DIR}')
print(f'  基础模型:  {BASE_MODEL}')
print(f'  训练步数:  {TRAIN_STEPS}')
print(f'  Batch 大小: {BATCH_SIZE}')
print(f'  服务端口:  {SERVER_PORT}')

if not os.path.isdir(DATA_DIR):
    print(f'\n⚠️  数据目录 {DATA_DIR} 不存在 → 跳过训练，仅部署推理服务')
    SKIP_TRAINING = True
elif not os.path.isfile(labels_path):
    print(f'\n⚠️  未找到 {labels_path} → 跳过训练')
    SKIP_TRAINING = True
else:
    import json
    with open(labels_path) as f:
        RECORD_COUNT = len(json.load(f))
    print(f'\n✅ 找到 {RECORD_COUNT} 条训练样本')
    if RECORD_COUNT < 10:
        print('  ⚠️  样本少于 10 条，建议至少准备 50 条以获得较好效果')

if os.path.isfile(os.path.join(MODEL_DIR, 'adapter_config.json')):
    print(f'\n✅ 发现已有 LoRA 权重（{MODEL_DIR}），将跳过训练直接部署')
    SKIP_TRAINING = True

print(f'\n  训练状态: {"⏭️  跳过" if SKIP_TRAINING else "✅ 将执行"}')

## 步骤 4 / 5 — LoRA 微调训练

> ⏳ RTX 4090 约 **15-20 分钟**；有已有权重或无数据时自动跳过

In [ ]:
if SKIP_TRAINING:
    print('⏭️  跳过训练步骤（已有权重或无训练数据）')
else:
    import json, os, torch
    from datasets import Dataset, Audio
    from transformers import (
        WhisperProcessor, WhisperForConditionalGeneration,
        Seq2SeqTrainingArguments, Seq2SeqTrainer
    )
    from peft import LoraConfig, get_peft_model

    print(f'📦 加载基础模型: {BASE_MODEL}')
    processor = WhisperProcessor.from_pretrained(BASE_MODEL, language=LANGUAGE, task='transcribe')
    model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL)

    # RTX 4090：优先使用 BF16
    USE_BF16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    if USE_BF16:
        model = model.to(torch.bfloat16)
        print('🎮 使用 BF16 精度（RTX 4090 原生支持）')
    else:
        print('⚠️  BF16 不支持，回退到 FP16')

    # LoRA 配置（针对 Whisper 注意力层）
    lora_config = LoraConfig(
        r=16, lora_alpha=32,
        target_modules=['q_proj', 'v_proj'],
        lora_dropout=0.05, bias='none'
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    # 加载数据
    print('\n📂 加载训练数据...')
    labels_path = os.path.join(DATA_DIR, 'labels.json')
    with open(labels_path) as f:
        records = json.load(f)
    for r in records:
        r['audio'] = os.path.join(DATA_DIR, r['audio'])

    dataset = Dataset.from_list(records).cast_column('audio', Audio(sampling_rate=16000))

    def preprocess(batch):
        audio = batch['audio']['array']
        inputs = processor(audio, sampling_rate=16000, return_tensors='pt')
        batch['input_features'] = inputs.input_features[0]
        batch['labels'] = processor.tokenizer(batch['text']).input_ids
        return batch

    print('⚙️  预处理数据...')
    dataset = dataset.map(preprocess, remove_columns=['audio', 'text'])
    split = dataset.train_test_split(test_size=0.1)

    # 训练参数（RTX 4090 优化）
    os.makedirs(MODEL_DIR, exist_ok=True)
    args = Seq2SeqTrainingArguments(
        output_dir=MODEL_DIR,
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=1,
        max_steps=TRAIN_STEPS,
        learning_rate=1e-3,
        warmup_steps=50,
        evaluation_strategy='steps',
        eval_steps=100,
        save_steps=100,
        logging_steps=25,
        predict_with_generate=True,
        bf16=USE_BF16,
        fp16=not USE_BF16,
        report_to='none',
        dataloader_num_workers=4,
        no_cuda=False,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=split['train'],
        eval_dataset=split['test'],
        tokenizer=processor.feature_extractor,
    )

    print(f'\n🚀 开始训练！batch_size={BATCH_SIZE}, steps={TRAIN_STEPS}')
    trainer.train()

    print('\n💾 保存 LoRA 权重...')
    model.save_pretrained(MODEL_DIR)
    processor.save_pretrained(MODEL_DIR)
    print(f'✅ 训练完成！权重已保存至 {MODEL_DIR}')

## 步骤 5 / 5 — 启动 FastAPI 推理服务

> 💡 JupyterLab 中服务以**后台线程**运行，不会阻塞 Notebook  
> 访问 `http://localhost:8000/health` 可确认服务状态

In [ ]:
import io, torch, threading, uvicorn, nest_asyncio, logging
import soundfile as sf
from fastapi import FastAPI, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel

# JupyterLab 必须：允许嵌套事件循环
nest_asyncio.apply()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('resonance')

# ── 初始化 FastAPI ──────────────────────────────────────────
app = FastAPI(title='Project Resonance ASR', version='1.0.0')
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_methods=['POST', 'GET'],
    allow_headers=['*'],
)

# ── 加载模型 ────────────────────────────────────────────────
print('🔄 正在加载模型...')
_processor = WhisperProcessor.from_pretrained(MODEL_DIR)
_base = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL)
_model = PeftModel.from_pretrained(_base, MODEL_DIR)
_model.eval()

if torch.cuda.is_available():
    _dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
    _model = _model.to('cuda', dtype=_dtype)
    print(f'✅ 模型已加载至 GPU（精度: {_dtype}）')
else:
    print('⚠️  GPU 不可用，使用 CPU 推理（速度较慢）')

# ── 接口定义 ────────────────────────────────────────────────
@app.get('/health')
def health():
    info = {'status': 'ok', 'model': BASE_MODEL}
    if torch.cuda.is_available():
        info.update({
            'gpu': torch.cuda.get_device_name(0),
            'memory_used_mb': round(torch.cuda.memory_allocated(0) / 1024**2, 1),
            'memory_total_mb': round(torch.cuda.get_device_properties(0).total_memory / 1024**2, 1),
        })
    return info

@app.post('/transcribe')
async def transcribe(file: UploadFile):
    try:
        data = await file.read()
        audio, sr = sf.read(io.BytesIO(data))
        if sr != 16000:
            import librosa
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        inputs = _processor(audio, sampling_rate=16000, return_tensors='pt').to(device)
        with torch.no_grad():
            ids = _model.generate(**inputs, language=LANGUAGE, task='transcribe')
        text = _processor.batch_decode(ids, skip_special_tokens=True)[0].strip()
        logger.info(f'识别结果: {text}')
        return {'text': text, 'success': True}
    except Exception as e:
        logger.error(f'识别失败: {e}')
        raise HTTPException(500, f'识别失败: {str(e)}')

# ── 后台启动服务（不阻塞 Notebook）──────────────────────────
config = uvicorn.Config(app, host='0.0.0.0', port=SERVER_PORT, log_level='info')
server = uvicorn.Server(config)

thread = threading.Thread(target=server.run, daemon=True)
thread.start()

print()
print('=' * 50)
print('  🎉 服务已启动！')
print('=' * 50)
print(f'  健康检查:  http://localhost:{SERVER_PORT}/health')
print(f'  识别接口:  POST http://localhost:{SERVER_PORT}/transcribe')
print()
print('  在 App 设置中填写公网地址：')
print(f'  http://<服务器公网IP>:{SERVER_PORT}/transcribe')
print()
print('  停止服务：重启此 Kernel 或关闭 JupyterLab 即可')

## 🧪 可选：本地快速测试识别效果

In [ ]:
import requests

# 修改为你的音频文件路径
TEST_AUDIO = '/root/data/test.wav'

if not __import__('os').path.isfile(TEST_AUDIO):
    print(f'⚠️  测试文件不存在：{TEST_AUDIO}')
    print('请修改 TEST_AUDIO 变量指向一个 WAV 音频文件')
else:
    with open(TEST_AUDIO, 'rb') as f:
        resp = requests.post(
            f'http://localhost:{SERVER_PORT}/transcribe',
            files={'file': ('test.wav', f, 'audio/wav')}
        )
    if resp.ok:
        result = resp.json()
        print(f'✅ 识别结果：{result["text"]}')
    else:
        print(f'❌ 请求失败：{resp.status_code} — {resp.text}')